In [1]:
# import core libraries and modules
import torch
import numpy as np
import os
import time
from tqdm import tqdm
import gc

# import file and dataset utilities
from glob import glob
from torch.utils.data import Dataset, DataLoader

# import model components and plotting tools
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)
if device.type == 'cuda':
    print("CUDA is available. GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA not available. Using CPU.")

Using: cuda
CUDA is available. GPU: NVIDIA GeForce RTX 4070


In [3]:
# load cube split lists and paths

# paths
data_dir = '../results/stacked_k8_dataset'
split_dir = '../results/split_k8_cube_lists'

# load train, val, and test cube name lists
train_cubes = np.load(os.path.join(split_dir, 'train_cubes.npy'))
val_cubes   = np.load(os.path.join(split_dir, 'val_cubes.npy'))
test_cubes  = np.load(os.path.join(split_dir, 'test_cubes.npy'))

# find all chunked X, y, and id files
X_paths = sorted(glob(os.path.join(data_dir, '*_X.npy')))
y_paths = sorted(glob(os.path.join(data_dir, '*_y.npy')))
id_paths = sorted(glob(os.path.join(data_dir, '*_ids.npy')))

# group by cube
cube_to_paths = {}
for x_path, y_path, id_path in zip(X_paths, y_paths, id_paths):
    cube = os.path.basename(x_path).replace('_X.npy', '')
    cube_to_paths[cube] = (x_path, y_path, id_path)

print(f"Found {len(cube_to_paths)} cubes in chunked format.")

Found 137 cubes in chunked format.


In [4]:
# RAM-safe dataset class

# dataset class that loads a fixed number of cubes fully into ram
class CRISMDatasetSubsetRAM(Dataset):
    def __init__(self, cube_names, cube_to_paths, max_cubes=None):
        self.X_arrays = []
        self.y_arrays = []
        self.starts = []
        count = 0

        # load only the first max_cubes cubes fully into memory
        selected_cubes = cube_names if max_cubes is None else cube_names[:max_cubes]
        print(f"[info] Loading {len(selected_cubes)} cubes into RAM...")
        
        for i, cube in enumerate(selected_cubes, start=1):
            x_path, y_path, _ = cube_to_paths[cube]
            X = np.load(x_path)  # fully into memory
            y = np.load(y_path)
            print(f"[{i}/{len(selected_cubes)}] [loaded] {cube}: {X.shape[0]} pixels")

            self.X_arrays.append(X)
            self.y_arrays.append(y)
            self.starts.append((count, count + len(y)))
            count += len(y)

        # total number of samples
        self.total_len = count
        print(f"[OK] Total loaded samples: {self.total_len}")

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        # locate which cube this index belongs to
        for i, (start, end) in enumerate(self.starts):
            if start <= idx < end:
                local_idx = idx - start
                X = self.X_arrays[i][local_idx]
                y = self.y_arrays[i][local_idx]
                return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)
        raise IndexError("[ERROR] Index out of bounds.")

In [5]:
# model definition

# simple 1d cnn for hyperspectral pixel classification
class SpectralCNN(nn.Module):
    def __init__(self, input_dim=469, num_classes=8):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=5, padding=2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, padding=2)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)        # reshape to [batch, 1, features]
        x = F.relu(self.conv1(x)) # apply first conv layer
        x = F.relu(self.conv2(x)) # apply second conv layer
        x = self.pool(x)          # global max pool
        x = x.squeeze(-1)         # flatten to [batch, 32]
        return self.fc(x)         # final prediction logits

In [6]:
# validation DataLoader

val_ds = CRISMDatasetSubsetRAM(val_cubes, cube_to_paths, max_cubes=len(val_cubes))
val_loader = DataLoader(val_ds, batch_size=40960, num_workers=18, pin_memory=True)

[info] Loading 28 cubes into RAM...
[1/28] [loaded] frt0000a09c_07_if166j_mtr3: 239494 pixels
[2/28] [loaded] hrl0001d976_07_if183j_mtr3: 97491 pixels
[3/28] [loaded] hrl0000bdda_07_if183j_mtr3: 50921 pixels
[4/28] [loaded] frt0001c558_07_if165j_mtr3: 88720 pixels
[5/28] [loaded] frt00017b1b_07_if166j_mtr3: 34470 pixels
[6/28] [loaded] frt000165f7_07_if166j_mtr3: 20378 pixels
[7/28] [loaded] frt00020c77_07_if166j_mtr3: 78947 pixels
[8/28] [loaded] frt0000bc1c_07_if166j_mtr3: 121553 pixels
[9/28] [loaded] frt00017d42_07_if166j_mtr3: 11358 pixels
[10/28] [loaded] frt000097e2_07_if166j_mtr3: 277231 pixels
[11/28] [loaded] hrl00013fd3_07_if183j_mtr3: 83462 pixels
[12/28] [loaded] frt000251c0_07_if165j_mtr3: 75669 pixels
[13/28] [loaded] frt0001821c_07_if166j_mtr3: 27598 pixels
[14/28] [loaded] frt0000805f_07_if166j_mtr3: 435657 pixels
[15/28] [loaded] frt00005c5e_07_if166j_mtr3: 460247 pixels
[16/28] [loaded] frt00003e12_07_if166j_mtr3: 444615 pixels
[17/28] [loaded] frt0000a9be_07_if165j_

In [7]:
# training loop over all train cubes

# initialize model, optimizer, and loss function
model = SpectralCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# training loop
epochs_per_block = 8
block_size = 12

# total training time timer
start_total_time = time.time()

for block_start in range(0, len(train_cubes), block_size):
    subset = train_cubes[block_start:block_start + block_size]
    print(f"\n Training on cubes {block_start} to {block_start + len(subset) - 1}")
    
    train_ds = CRISMDatasetSubsetRAM(subset, cube_to_paths)
    train_loader = DataLoader(train_ds, batch_size=40960, shuffle=True, num_workers=18, pin_memory=True)

    # start block timer
    block_start_time = time.time()
    
    for epoch in range(1, epochs_per_block + 1):
        model.train()
        total_loss = 0
        start_time = time.time()

        # tqdm adds a live progress bar with ETA and speed
        for X_batch, y_batch in tqdm(train_loader, desc=f"epoch {epoch} [train block {block_start}]"):
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # run validation
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                preds = model(X_batch).argmax(dim=1)
                correct += (preds == y_batch).sum().item()
                total += y_batch.size(0)

        val_acc = correct / total
        duration = time.time() - start_time
        print(f"[OK] Block {block_start} | Epoch {epoch} | Loss: {total_loss:.2f} | Val Acc: {val_acc:.4f} | Time: {duration:.1f}s")

    
    # clear memory after block training
    # delete dataset and subset references
    del train_ds
    del subset
    # force garbage collection
    gc.collect()
    
    # block timer end
    block_end_time = time.time()
    block_duration = block_end_time - block_start_time
    print(f"[OK] Block {block_start} completed in {block_duration:.1f}s")

# total learning time
total_learning_duration = time.time() - start_total_time
print(f"\n[OK] Total Training Time: {total_learning_duration:.1f}s")


 Training on cubes 0 to 11
[info] Loading 12 cubes into RAM...
[1/12] [loaded] frt00003584_07_if166j_mtr3: 427754 pixels
[2/12] [loaded] hrl0000d0c7_07_if183j_mtr3: 152370 pixels
[3/12] [loaded] frt0000a4b5_07_if167j_mtr3: 247912 pixels
[4/12] [loaded] frt00023565_07_if166j_mtr3: 41770 pixels
[5/12] [loaded] hrs0000b6a2_07_if176j_mtr3: 30724 pixels
[6/12] [loaded] hrl000141d5_07_if183j_mtr3: 70534 pixels
[7/12] [loaded] hrl00010eea_07_if183j_mtr3: 67812 pixels
[8/12] [loaded] frt00018781_07_if165j_mtr3: 33302 pixels
[9/12] [loaded] frt0001ecba_07_if166j_mtr3: 103203 pixels
[10/12] [loaded] frt00018524_07_if166j_mtr3: 100869 pixels
[11/12] [loaded] hrl00010d72_07_if183j_mtr3: 83983 pixels
[12/12] [loaded] frt000066a4_07_if166j_mtr3: 269502 pixels
[OK] Total loaded samples: 1629735


epoch 1 [train block 0]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:10<00:00,  3.90it/s]


[OK] Block 0 | Epoch 1 | Loss: 81.68 | Val Acc: 0.1667 | Time: 23.1s


epoch 2 [train block 0]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:09<00:00,  4.29it/s]


[OK] Block 0 | Epoch 2 | Loss: 77.38 | Val Acc: 0.2092 | Time: 22.4s


...

[OK] Block 72 | Epoch 6 | Loss: 12.47 | Val Acc: 0.8371 | Time: 21.2s


epoch 7 [train block 72]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:08<00:00,  3.94it/s]


[OK] Block 72 | Epoch 7 | Loss: 12.33 | Val Acc: 0.8374 | Time: 21.7s


epoch 8 [train block 72]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:08<00:00,  4.14it/s]


[OK] Block 72 | Epoch 8 | Loss: 12.21 | Val Acc: 0.8399 | Time: 21.2s
[OK] Block 72 completed in 170.9s

[OK] Total Training Time: 1805.8s


In [8]:
# test prediction with timing

# load all test cubes into RAM
test_ds = CRISMDatasetSubsetRAM(test_cubes, cube_to_paths, max_cubes=len(test_cubes))
# dataloader for test prediction
test_loader = DataLoader(test_ds, batch_size=40960, num_workers=18, pin_memory=True)

# evaluate model and store predictions
model.eval()
predictions = []

start_time = time.time()
with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.append(preds)

predictions = np.concatenate(predictions)
duration = time.time() - start_time
print(f"Done: predicted {len(predictions):,} test pixels in {duration:.1f}s")

[info] Loading 28 cubes into RAM...
[1/28] [loaded] frt0000c202_07_if165j_mtr3: 201990 pixels
[2/28] [loaded] frt0000d6d6_07_if166j_mtr3: 141886 pixels
[3/28] [loaded] frt000161ef_07_if167j_mtr3: 93605 pixels
[4/28] [loaded] hrs00012aa7_07_if175j_mtr3: 45132 pixels
[5/28] [loaded] hrl0001b769_07_if183j_mtr3: 101887 pixels
[6/28] [loaded] frt00021da6_07_if166j_mtr3: 43045 pixels
[7/28] [loaded] hrl000116c6_07_if183j_mtr3: 118360 pixels
[8/28] [loaded] frt00009c31_07_if166j_mtr3: 212648 pixels
[9/28] [loaded] frt0000bfd1_07_if166j_mtr3: 156969 pixels
[10/28] [loaded] frt000093be_07_if166j_mtr3: 288048 pixels
[11/28] [loaded] frt00008389_07_if166j_mtr3: 380642 pixels
[12/28] [loaded] frt000174f4_07_if166j_mtr3: 47459 pixels
[13/28] [loaded] frt0000bec0_07_if165j_mtr3: 131165 pixels
[14/28] [loaded] frt00011d4c_07_if166j_mtr3: 430661 pixels
[15/28] [loaded] hrl0000cc16_07_if184j_mtr3: 122464 pixels
[16/28] [loaded] frt0000c968_07_if166j_mtr3: 171121 pixels
[17/28] [loaded] hrs00011c01_07_i

In [9]:
# rebuild labelmaps

# paths
preprocessed_dir = '../data/nili_fossae_preprocessed_spectra'
output_dir = '../outputs/cnn_labelmaps'
os.makedirs(output_dir, exist_ok=True)

# get files
test_ids = np.concatenate([np.load(cube_to_paths[cube][2]) for cube in test_cubes])


# Reconstruct labelmaps
offset = 0
for cube in test_cubes:
    index_path = os.path.join(preprocessed_dir, f'{cube}_indices.npy')
    indices = np.load(index_path)
    h, w = indices[:, 0].max()+1, indices[:, 1].max()+1
    labelmap = np.full((h, w), -1, dtype=np.int16)

    preds_slice = predictions[offset:offset + len(indices)]
    for (r, c), label in zip(indices, preds_slice):
        labelmap[r, c] = label

    offset += len(indices)

    # save
    np.save(os.path.join(output_dir, f"{cube}_cnn_labelmap.npy"), labelmap)

    # show PNG
    plt.figure(figsize=(6, 6))
    plt.imshow(labelmap, cmap='tab10', interpolation='nearest')
    plt.axis('off')
    plt.title(f"{cube} - CNN")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{cube}_cnn_labelmap.png"))
    plt.close()

    print(f"[OK] Saved: {cube} labelmap")

print("All CNN labelmaps saved to:", output_dir)


[OK] Saved: frt0000c202_07_if165j_mtr3 labelmap
[OK] Saved: frt0000d6d6_07_if166j_mtr3 labelmap
[OK] Saved: frt000161ef_07_if167j_mtr3 labelmap
[OK] Saved: hrs00012aa7_07_if175j_mtr3 labelmap
[OK] Saved: hrl0001b769_07_if183j_mtr3 labelmap
[OK] Saved: frt00021da6_07_if166j_mtr3 labelmap
[OK] Saved: hrl000116c6_07_if183j_mtr3 labelmap
[OK] Saved: frt00009c31_07_if166j_mtr3 labelmap
[OK] Saved: frt0000bfd1_07_if166j_mtr3 labelmap
[OK] Saved: frt000093be_07_if166j_mtr3 labelmap
[OK] Saved: frt00008389_07_if166j_mtr3 labelmap
[OK] Saved: frt000174f4_07_if166j_mtr3 labelmap
[OK] Saved: frt0000bec0_07_if165j_mtr3 labelmap
[OK] Saved: frt00011d4c_07_if166j_mtr3 labelmap
[OK] Saved: hrl0000cc16_07_if184j_mtr3 labelmap
[OK] Saved: frt0000c968_07_if166j_mtr3 labelmap
[OK] Saved: hrs00011c01_07_if175j_mtr3 labelmap
[OK] Saved: frt0000c62b_07_if166j_mtr3 labelmap
[OK] Saved: frt00007bc8_07_if166j_mtr3 labelmap
[OK] Saved: frt00017103_07_if165j_mtr3 labelmap
[OK] Saved: frt00016655_07_if166j_mtr3 l